In [1]:
from typing import get_args

def build_tree(model, indent: int = 0) -> str:
    """모델 구조를 트리 형태 문자열로 반환"""
    lines = []
    for name, field in model.model_fields.items():
        type_ = field.annotation
        desc = field.description or ""
        prefix = "  " * indent + f"- {name} ({type_.__name__ if hasattr(type_, '__name__') else type_})"
        if desc:
            prefix += f": {desc}"
        lines.append(prefix)

        # Nested BaseModel
        if hasattr(type_, "model_fields"):
            lines.append(build_tree(type_, indent + 1))

        # List of BaseModel
        elif str(type_).startswith("list") and hasattr(get_args(type_)[0], "model_fields"):
            inner = get_args(type_)[0]
            lines.append(build_tree(inner, indent + 1))
    return "\n".join(lines)

In [2]:
from langchain_openai import ChatOpenAI
import uuid,  json
from langchain_community.chat_message_histories import ChatMessageHistory
from langchain_core.runnables import RunnablePassthrough, RunnableLambda
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder



# ------------------
from pydantic import BaseModel, Field

class GeneralMember(BaseModel):
    """팀 구성 현황(대표자 본인 제외)의 한 행"""
    position: str = Field(default="", description="직위")
    task: str = Field(default="", description="담당 업무")
    capability: str = Field(default="", description="보유 역량")
    status: str = Field(default="", description="구성 상태")

class GeneralInfo(BaseModel):
    """사업계획서 - 일반현황 섹션"""
    item_name: str = Field(default="", description="사업아이템명")
    deliverables: str = Field(default="", description="산출물(협약기간 내 목표)")
    job: str = Field(default="", description="직업(직장명 기재 불가)")
    company_name: str = Field(default="", description="기업(예정)명")
    general_members: list[GeneralMember] = Field(
        default_factory=list,
        description="팀 구성 현황 리스트(대표자 본인 제외)"
    )

class ProblemPoint(BaseModel):
    """국내·외 시장 현황 및 문제점 등"""
    main: str = Field(
        default="",
        description="주요 문제점 (시장 현황, 기술적/산업적 한계 등)"
    )
    sub: str = Field(
        default="",
        description="세부 문제점 또는 설명"
    )

class ProblemPointsInfo(BaseModel):
    """사업계획서 - 개발 아이템 소개 및 문제점 섹션"""
    problem_points: list[ProblemPoint] = Field(
        default_factory=list,
        description=(
            "개발하고자 하는 아이템의 국내·외 시장 현황 및 문제점 제시, "
            "이를 해결하기 위한 아이템 개발 필요성 기재"
        )
    )

class SolutionPoint(BaseModel):
    """아이디어를 제품·서비스로 개발 및 구체화하는 계획"""
    main: str = Field(
        default="",
        description="주요 계획 (예: 프로토타입 개발, 플랫폼 고도화 등)"
    )
    sub: str = Field(
        default="",
        description=(
            "세부 계획 (예: 일정, 기능·성능 차별성, 경쟁력 확보 전략, "
            "정부지원사업비 및 자기부담사업비 집행 계획 등)"
        )
    )

class Schedule(BaseModel):
    """사업기간 내 일정 세부 계획"""
    task: str = Field(
        default="",
        description="추진 내용"
    )
    period: str = Field(
        default="",
        description="추진 기간"
    )
    detail: str = Field(
        default="",
        description="세부 내용"
    )

class SolutionPointsInfo(BaseModel):
    """사업계획서 - 개발 계획 및 경쟁력 전략 섹션"""
    solution_points: list[SolutionPoint] = Field(
        default_factory=list,
        description=(
            "아이디어를 제품·서비스로 개발 또는 구체화하는 일정 및 계획, "
            "개발 아이템의 차별성과 경쟁력 확보 전략, "
            "정부지원사업비 및 자기부담사업비 집행 계획을 포함"
        )
    )
    schedules: list[Schedule] = Field(
        default_factory=list,
        description="사업기간 내 일정별 세부 계획"
    )

class Budget(BaseModel):
    """사업비 집행 계획의 한 행"""
    category: str = Field(
        default="",
        description="비목"
    )
    plan: str = Field(
        default="",
        description="집행 계획"
    )
    gov_support: str = Field(
        default="",
        description="정부지원 사업비 금액"
    )
    self_funding: str = Field(
        default="",
        description="자기부담 사업비 금액"
    )
    total: str = Field(
        default="",
        description="총 사업비 금액"
    )

class BudgetInfo(BaseModel):
    """사업계획서 - 사업비 집행 계획 섹션"""
    budgets: list[Budget] = Field(
        default_factory=list,
        description="사업비 집행 계획 리스트"
    )

class Strategy(BaseModel):
    """아이템의 기능·성능 차별성 및 경쟁력 확보 전략"""
    main: str = Field(
        default="",
        description="주요 전략"
    )
    sub: str = Field(
        default="",
        description="세부 전략 또는 실행 방법"
    )

class ScaleupSchedule(BaseModel):
    """아이템 사업화·확산(Scale-up) 일정"""
    task: str = Field(
        default="",
        description="주요 업무 또는 활동"
    )
    period: str = Field(
        default="",
        description="수행 기간"
    )
    detail: str = Field(
        default="",
        description="세부 설명"
    )

class StrategyInfo(BaseModel):
    """사업계획서 - 실현 가능성 및 Scale-up 전략 섹션"""
    strategies: list[Strategy] = Field(
        default_factory=list,
        description="아이템의 기능·성능 차별성 및 경쟁력 확보 전략 리스트"
    )
    scaleup_schedules: list[ScaleupSchedule] = Field(
        default_factory=list,
        description="아이템 사업화·확산(Scale-up) 추진 일정"
    )

class TeamPoint(BaseModel):
    """대표자 및 팀원 구성 계획의 주요 항목"""
    main: str = Field(
        default="",
        description="주요 항목"
    )
    sub: str = Field(
        default="",
        description="세부 설명"
    )

class PlanMember(BaseModel):
    """팀 구성(대표자 제외)"""
    position: str = Field(
        default="",
        description="직위"
    )
    task: str = Field(
        default="",
        description="담당 업무"
    )
    capability: str = Field(
        default="",
        description="보유 역량"
    )
    status: str = Field(
        default="",
        description="구성 상태"
    )

class Partner(BaseModel):
    """협력 기관 현황 및 협업 방안"""
    name: str = Field(
        default="",
        description="협력 기관명"
    )
    capability: str = Field(
        default="",
        description="기관이 보유한 역량"
    )
    plan: str = Field(
        default="",
        description="협업 방안"
    )
    period: str = Field(
        default="",
        description="협력 시기"
    )

class TeamInfo(BaseModel):
    """사업계획서 - 팀 구성 및 협력 기관 섹션"""
    team_points: list[TeamPoint] = Field(
        default_factory=list,
        description="대표자 및 팀원 구성 계획의 주요 항목 리스트"
    )
    plan_members: list[PlanMember] = Field(
        default_factory=list,
        description="팀 구성(대표자 제외)"
    )
    partners: list[Partner] = Field(
        default_factory=list,
        description="협력 기관 현황 및 협업 방안 리스트"
    )

class Summary(BaseModel):
    """아이템 개요(요약)"""
    name: str = Field(
        default="",
        description="아이템 명칭"
    )
    category: str = Field(
        default="",
        description="아이템 범주"
    )
    item_overview: str = Field(
        default="",
        description="아이템 개요: 제품·서비스의 사용 용도, 사용 가격, 핵심 기능·성능, 고객 제공 혜택 등"
    )
    problem_summary: str = Field(
        default="",
        description="문제 인식: 국내·외 시장 현황 및 문제점, 이를 해결하기 위한 창업 아이템 필요성"
    )
    solution_summary: str = Field(
        default="",
        description="실현 가능성: 사업기간 내 개발 계획, 차별성 및 경쟁력 확보 전략"
    )
    scaleup_summary: str = Field(
        default="",
        description="성장 전략: 경쟁사 분석, 시장 진입 전략, 사업 모델, 로드맵 및 투자유치 전략"
    )
    team_summary: str = Field(
        default="",
        description="팀 구성: 대표자, 팀원, 협력 파트너 등 역할과 활용 계획"
    )

class BusinessPlanWriter:
    PROMPTS = {
        "system": "너는 전문 사업 컨설턴트이자 사업계획서 작성 전문가이다. 스타트업, 중소기업, 투자 유치용 사업계획서 작성에 특화되어 있으며, 사용자가 제공하는 정보와 요구사항을 바탕으로 체계적이고 설득력 있는 사업계획서를 작성하는 역할을 수행한다.",
        GeneralInfo: {
            "user": f'''스키마에서 이번에 채워야 할 필드는
{build_tree(GeneralInfo)}
이다.

[필드별 작성 가이드 & 예시]
- 사업아이템명: "OO기술이 적용된 OO기능의(혜택을 제공하는) OO제품·서비스 등"
- 산출물(협약기간 내 목표): "모바일 어플리케이션(0개), 웹사이트(0개)"
- 직업(직장명 기재 불가): "교수 / 연구원 / 사무직 / 일반인 / 대학생 등"
- 기업(예정)명

[팀 구성 현황 리스트(대표자 본인 제외) - 각 행의 예시]
- 직위: "공동대표", "대리" 등
- 담당 업무: "S/W 개발 총괄", "홍보 및 마케팅" 등
- 보유 역량: "OO학 학사", "OO 관련 경력(00년 이상)"
- 구성 상태: "완료", "예정"

[작성 규칙]
1) 반드시 한국어로 작성.

[검색된 참고자료]
- 회사DB: None
- 공고문DB: None


위 정보를 참고하여 {list(GeneralInfo.model_fields.keys())} 항목을 작성하라.'''
        },

        ProblemPointsInfo: {
            "user": f'''스키마에서 이번에 채워야 할 필드는
{build_tree(ProblemPointsInfo)}
이다.

개발하고자 하는 아이템의 국내·외 시장 현황 및 문제점 등의 제시
문제 해결을 위한 아이템의 개발 필요성 등 기재_개발 아이템 소개


[작성 규칙]
1) 반드시 한국어로 작성.

[검색된 참고자료]
- 회사DB: None
- 공고문DB: None

위 정보를 참고하여 {list(ProblemPointsInfo.model_fields.keys())} 항목을 작성하라.'''
        },

        SolutionPointsInfo: {
            "user": f'''스키마에서 이번에 채워야 할 필드는
{build_tree(SolutionPointsInfo)}
이다.

[필드별 작성 가이드 & 예시]
- 주요 계획(main): "모바일 앱 서비스 프로토타입 개발(1년차 완료)", "클라우드 기반 플랫폼 고도화(2년차 진행)" 등
- 세부 계획(sub): "UI/UX 시제품 제작 후 베타 테스트 실시", "정부지원사업비는 인건비 및 장비구축에 활용, 자기부담사업비는 마케팅 비용에 배정" 등

[사업기간 내 일정(schedules) - 각 행의 예시]
- 추진 업무(task): "필수 개발 인력 채용", "제품 패키지 디자인", "홍보용 웹사이트 제작", "시제품 완성" 등
- 수행 기간(period): "00.00 ~ 00.00", "협약기간 말" 등
- 세부 설명(detail): "OO 전공 경력 직원 00명 채용", "제품 패키지 디자인 용역 진행", "웹사이트 자체 제작", "협약기간 내 시제품 제작 완료" 등


[작성 규칙]
1) 반드시 한국어로 작성.

[검색된 참고자료]
- 회사DB: None
- 공고문DB: None

위 정보를 참고하여 {list(SolutionPointsInfo.model_fields.keys())} 항목을 작성하라.'''
        },

        BudgetInfo: {
            "user": f'''스키마에서 이번에 채워야 할 필드는
{build_tree(BudgetInfo)}
이다.

[필드별 작성 가이드 & 예시]
- 비목(category): "재료비", "외주용역비", "인건비" 등
- 집행 계획(plan): "DMD소켓 구입(00개x0000원)", "진입대류 구입(00개x000원)", "시급협회와 외주용역" 등
- 정부지원 사업비(gov_support): "3,448,000"
- 자기부담 사업비(self_funding): "7,000,000(현금)", "2,000,000(현물)" 등
- 총 사업비(total): "10,000,000"

[작성 규칙]
1) 반드시 한국어로 작성.

[검색된 참고자료]
- 회사DB: None
- 공고문DB: None

위 정보를 참고하여 {list(BudgetInfo.model_fields.keys())} 항목을 작성하라.'''
        },

        StrategyInfo: {
            "user": f'''스키마에서 이번에 채워야 할 필드는
{build_tree(StrategyInfo)}
이다.

[필드별 작성 가이드 & 예시]
- 주요 전략(main): "제품 성능 고도화", "해외 시장 진출", "지속적 업데이트를 통한 경쟁력 유지" 등
- 세부 전략(sub): "국내외 특허 확보", "품질 인증 취득", "해외 파트너사 협력 체계 구축" 등

[사업화·확산 일정(scaleup_schedules) - 각 행의 예시]
- 추진 업무(task): "필수 개발 인력 채용", "제품 패키지 디자인", "홍보용 웹사이트 제작", "시제품 완성" 등
- 수행 기간(period): "00.00 ~ 00.00", "협약기간 말" 등
- 세부 설명(detail): "OO 전공 경력 직원 00명 채용", "제품 패키지 디자인 용역 진행", "웹사이트 자체 제작", "협약기간 내 시제품 제작 완료" 등


[작성 규칙]
1) 반드시 한국어로 작성.

[검색된 참고자료]
- 회사DB: None
- 공고문DB: None

위 정보를 참고하여 {list(StrategyInfo.model_fields.keys())} 항목을 작성하라.'''
        },

        TeamInfo: {
            "user": f'''스키마에서 이번에 채워야 할 필드는
{build_tree(TeamInfo)}
이다.

[필드별 작성 가이드 & 예시]
- 대표 및 팀원 주요 항목(team_points):
  - main: "대표자의 경력과 전문성", "팀원들의 보유 역량"
  - sub: "OO학 박사, OO과 교수 경력(00년)", "정부지원사업 수행 경험, 장비·시설 보유" 등

- 팀 구성(plan_members):
  - position: "공동대표", "대리" 등
  - task: "S/W 개발 총괄", "홍보 및 마케팅" 등
  - capability: "OO학 박사, OO 관련 경력(00년 이상)" 등
  - status: "완료(00.00)", "예정(00.00)" 등

- 협력 기관(partners):
  - name: "○○전자", "○○기업" 등
  - capability: "시제품 관련 H/W 제작·개발", "S/W 제작·개발" 등
  - plan: "테스트 장비 지원", "웹사이트 제작 용역" 등
  - period: "00.00 ~", "협약기간 내" 등


[작성 규칙]
1) 반드시 한국어로 작성.

[검색된 참고자료]
- 회사DB: None
- 공고문DB: None

위 정보를 참고하여 {list(TeamInfo.model_fields.keys())} 항목을 작성하라.'''
        },

        Summary: {
            "user": f'''스키마에서 이번에 채워야 할 필드는
{build_tree(Summary)}
이다.

[필드별 작성 가이드 & 예시]
- 명칭(name): "게토레이", "Windows", "알파고" 등
- 범주(category): "스포츠음료", "OS(운영체제)", "인공지능프로그램" 등
- 아이템 개요(item_overview): "OO기술이 적용된 OO제품으로, OO기능과 OO혜택 제공" 등
- 문제 인식(problem_summary): "국내 시장에서 ○○ 기술 보급률 부족, 소비자 접근성 한계" 등
- 실현 가능성(solution_summary): "사업기간 내 모바일 앱 프로토타입 개발, 기능 차별화 전략 수립" 등
- 성장 전략(scaleup_summary): "경쟁사 대비 ○○ 차별성, 글로벌 시장 진출 로드맵, 투자유치 계획" 등
- 팀 구성(team_summary): "대표자와 공동대표, 마케팅 담당자, 협력기관 ○○전자와의 협업 계획" 등


[작성 규칙]
1) 반드시 한국어로 작성.

[검색된 참고자료]
- 회사DB: None
- 공고문DB: None

위 정보를 참고하여 {list(Summary.model_fields.keys())} 항목을 작성하라.'''
        },
    }

    _store = {}

    # _default_llm = ChatOpenAI(model="gpt-5")
    _default_llm = ChatOpenAI(
        openai_api_key="EMPTY",
        openai_api_base="http://localhost:8000/v1",
        model="K-intelligence/Midm-2.0-Base-Instruct",
    )

    def __init__(self, llm=None, session_id=None):
        self.llm = llm or self._default_llm
        self.session_id = session_id or str(uuid.uuid4())
        self.history = self._get_session_history()

        self.general_chain = self._make_chain(GeneralInfo)
        self.problem_chain = self._make_chain(ProblemPointsInfo)
        self.solution_chain = self._make_chain(SolutionPointsInfo)
        self.budget_chain = self._make_chain(BudgetInfo)
        self.scaleup_chain = self._make_chain(StrategyInfo)
        self.team_chain = self._make_chain(TeamInfo)
        self.summary_chain = self._make_chain(Summary)

        # self.full_chain = (
        #     {
        #         "context": RunnablePassthrough(),
        #         "chat_history": lambda _: self.history.messages,
        #     }
        #     | {
        #         "general": self.general_chain,
        #         "problem": self.problem_chain,
        #         "solution": self.solution_chain,
        #         "scaleup": self.scaleup_chain,
        #         "team": self.team_chain,
        #     }
        #     | RunnableLambda(
        #         lambda parts: self.summary_chain.invoke({
        #             "context": (
        #                 f"[문제]\n{parts['problem'].json()}\n\n"
        #                 f"[솔루션]\n{parts['solution'].json()}\n\n"
        #                 f"[성장전략]\n{parts['scaleup'].json()}\n\n"
        #                 f"[팀]\n{parts['team'].json()}"
        #             ),
        #             "chat_history": self.history.messages
        #         })
        #     )
        # )

    def _get_session_history(self) -> ChatMessageHistory:
        if self.session_id not in self._store:
            self._store[self.session_id] = ChatMessageHistory()
        return self._store[self.session_id]
    
    def _make_chain(self, schema):
        system_msg = self.PROMPTS["system"]
        user_msg = self.PROMPTS[schema]["user"]
        prompt = ChatPromptTemplate.from_messages(
            [
                (
                    "system", system_msg
                ),
                MessagesPlaceholder(variable_name="chat_history"),
                (
                    "user", user_msg
                ), 
            ]
        )
        return prompt | self.llm.with_structured_output(schema)
    
    # 개별 실행 메서드
    def write_general_status(self):
        request = f"{list(GeneralInfo.model_fields.keys())} 항목을 작성하라."
        out = self.general_chain.invoke({"request": request, "chat_history": self.history.messages})
        self.history.add_user_message(request)
        self.history.add_ai_message(out.model_dump_json())
        return out

    def write_problem(self):
        request = f"{list(ProblemPointsInfo.model_fields.keys())} 항목을 작성하라."
        out = self.problem_chain.invoke({"request": request, "chat_history": self.history.messages})
        self.history.add_user_message(request)
        self.history.add_ai_message(out.model_dump_json())
        return out

    def write_solution(self):
        request = f"{list(SolutionPointsInfo.model_fields.keys())} 항목을 작성하라."
        out = self.solution_chain.invoke({"request": request, "chat_history": self.history.messages})
        self.history.add_user_message(request)
        self.history.add_ai_message(out.model_dump_json())
        return out

    def write_budget(self):
        request = f"{list(BudgetInfo.model_fields.keys())} 항목을 작성하라."
        out = self.budget_chain.invoke({"request": request, "chat_history": self.history.messages})
        self.history.add_user_message(request)
        self.history.add_ai_message(out.model_dump_json())
        return out

    def write_scaleup(self):
        request = f"{list(StrategyInfo.model_fields.keys())} 항목을 작성하라."
        out = self.scaleup_chain.invoke({"request": request, "chat_history": self.history.messages})
        self.history.add_user_message(request)
        self.history.add_ai_message(out.model_dump_json())
        return out

    def write_team(self):
        request = f"{list(TeamInfo.model_fields.keys())} 항목을 작성하라."
        out = self.team_chain.invoke({"request": request, "chat_history": self.history.messages})
        self.history.add_user_message(request)
        self.history.add_ai_message(out.model_dump_json())
        return out

    def write_summary(self):
        request = f"{list(Summary.model_fields.keys())} 항목을 작성하라."
        out = self.summary_chain.invoke({"request": request, "chat_history": self.history.messages})
        self.history.add_user_message(request)
        self.history.add_ai_message(out.model_dump_json())
        return out

    # 전체 실행 (full_chain 활용)
    # def run_all(self, context: str, output_file: str = "business_plan.json"):
    #     result = self.full_chain.invoke(context)

    #     # JSON 직렬화
    #     if isinstance(result, Summary):  # 마지막 단계가 Summary 객체
    #         result_dict = {"summary": result.dict()}
    #     else:
    #         result_dict = result

    #     with open(output_file, "w", encoding="utf-8") as f:
    #         json.dump(result_dict, f, ensure_ascii=False, indent=2)

    #     return result_dict


In [3]:
writer = BusinessPlanWriter()

In [4]:
problem = {}
problem.update(writer.write_general_status().model_dump())

In [5]:
problem.update(writer.write_problem().model_dump())

In [6]:
problem.update(writer.write_solution().model_dump())

In [7]:
problem.update(writer.write_budget().model_dump())

In [8]:
problem.update(writer.write_scaleup().model_dump())

In [9]:
problem.update(writer.write_team().model_dump())

In [10]:
problem.update(writer.write_summary().model_dump())

In [13]:
from docxtpl import DocxTemplate
from pathlib import Path

docx_path = Path("사업계획서_양식.docx")

doc = DocxTemplate(docx_path)
context = problem

doc.render(context)

modified_docx_path = docx_path.with_stem("사업계획서")

doc.save(modified_docx_path)

In [12]:
problem

{'item_name': 'AI 기반 맞춤형 여행 일정 추천 서비스',
 'deliverables': '모바일 어플리케이션 1개, 웹사이트 1개',
 'job': '연구원',
 'company_name': '트래블AI 솔루션즈',
 'general_members': [{'position': 'CTO',
   'task': '백엔드 시스템 개발 및 데이터 처리',
   'capability': '컴퓨터공학 석사, 10년 이상 IT 개발 경력',
   'status': '예정'},
  {'position': '마케팅 매니저',
   'task': '디지털 마케팅 전략 수립 및 실행',
   'capability': '경영학 학사, 5년 이상 마케팅 경험',
   'status': '완료'}],
 'problem_points': [{'main': '국내·외 여행 시장은 매년 성장하고 있으나, 기존 여행상품은 획일화된 패키지 위주로 제공되어 개인의 취향·일정·예산 등 다양한 니즈를 충분히 반영하지 못함. 자유여행 수요 증가에도 불구하고, 개별 여행자가 직접 일정을 짜는 데에는 시간·정보 부족 등 현실적 한계 존재.',
   'sub': '특히, 자유여행객은 여행지별 최적 일정 추천, 동선 최적화, 현지 맞춤 정보 제공 등에서 어려움을 겪으며, 정보의 신뢰성·실시간성 부족도 문제로 지적됨. 기존 플랫폼은 단순 검색·예약 기능에 집중, AI 기반 개인화 추천 서비스는 아직 초기 단계임.'},
  {'main': '기존 여행 일정 추천 서비스는 데이터 분석 및 알고리즘 활용이 미흡해, 실제 여행자의 선호도·상황 변화에 유연하게 대응하지 못함. 또한, 실시간 예약 연동이나 다국어 지원, 지역별 특화 콘텐츠 등 세부 기능이 부족함.',
   'sub': '여행업계 내에서도 AI·빅데이터를 활용한 맞춤형 서비스 도입이 더딘 상황이며, 여행자 중심의 직관적이고 자동화된 일정 설계 솔루션에 대한 시장 수요가 높아지고 있음.'}],
 'solution_points': [{'main'